# Convex Optimization Project Proposal
This notebook describes a convex optimization problem to be studied in a one-week project as part of the course Convex Optimization.

The optimization problem could be characterized as *covariance matrix fitting* where measured data, given by a  covariance matrix, is used to estimate a source covariance matrix through a propagation model.

Consider a measured (hermitian) covariance matrix $P\in\mathbb{C}^{M\times M}$ and a sound propagation model defined by transfer functions 

$$g_{m,n} = \frac{r_{0,n}}{r_{m,n}}e^{i\omega (r_{0,n}-r_{m,n})/c}\quad m=1,...,M,\;n=1,...,N, $$

where $\omega = 2\pi f$ is the angular frequency, $c = 343 m/s$ is the speed of sound, $r_{0,n}$ is the distance from the center of the microphone array to the $n$'th source point, and $r_{m,n}$ is the distance between the $m$'th microphone and $n$'th source point. The transfer functions take into account the propagation of sound from the source to the microphone. These are arranged in a tranfer matrix $G\in\mathbb{C}^{N\times M}$. The model can be decribed by

$$P = GXG^H$$

where $X\in \mathbb{C}^{N\times N}$ is a covariance matrix at the source plane which is to be estimated. It is sparse and ill-conditioned.

The optimization problem can be stated as a SDP:

$$\begin{align} 
\begin{aligned} 
\underset{X}{\mathrm{minimize}}\quad & \|P-GXG^H\|_F^2\\ 
\mathrm{subject \ to} \quad & X\succeq 0\\ 
\end{aligned} 
\end{align}
$$

Several options for the solution can be persued:
* $X$ is diagonal. In this case the number of variables can be reduced greatly, however, some work is required to ensure that $X*$ is in fact diagonal.
* First-order methods and warmstart. The optimization problem should be solved multiple times for different $\omega$'s. Exploit warmstart capability of first-order methods and the assumption that covariance matrices does not change much between similar $\omega$'s.
* Reduce problem by introducing a matrix $M$ of same dimensions as $P$, such that $X = G^HMG $, e.g.,

$$
\begin{align} 
\begin{aligned} 
\underset{M}{\mathrm{minimize}}\quad & \|P-GG^HMGG^H\|_F^2\\ 
\mathrm{subject \ to} \quad & M\succeq 0\\ 
\end{aligned} 
\end{align}
$$

and using $e_i^TG^HMGe_i = \sum_i tr(MGe_ie_iG^H)$, where $e_i$ is a unit-vector. Use resulting $M$ to obtain $X$.
* Adding regularization terms, e.g.

$$
\begin{align} 
\begin{aligned} 
\underset{X}{\mathrm{minimize}}\quad & \|P-GXG^H\|_F^2+\lambda\|X-G^HMG\|_F^2+\eta\|diag(X)\|_1\\ 
\mathrm{subject \ to} \quad & M\succeq 0\\ 
\end{aligned} 
\end{align}
$$

## Background

The generation of electricity from renewable energy sources is more important than ever following the global agreement signed by 195 countries at the COP21 climate conference in Paris, December 2015. The long term goal, to limit the global average temperature increase to 2$^{\circ}$C, requires governments around the world to limit their dependence on fossil fuels and adapt renewable energy sources.

This goal requires development and installation of more wind turbines (onshore and offshore) in the following years. Noise regulations on wind turbines near urban areas can restrict the installation of new wind turbines and limit the potential energy production. The main noise from wind turbines is aerodynamic and arises from turbulence caused by the air flow around the wind turbine blades in motion. The aerodynamic noise increases with the size of the wind turbine and with a demand for more productive, and thus larger wind turbines, there is an increased interest in quantifying the aerodynamic noise from wind turbine blades. In the design stage, wind turbine blades undergo a wealth of tests and simulations to ensure that the specific design is efficient. A wind tunnel is used to test the aerodynamic properties by applying a controlled airflow to a section of the blade. The wind tunnel can also be used to localize and quantify the noise sources with a microphone array, however, this requires specially designed wind tunnels.

In typical experimental setups, a wind turbine wing section, i.e., airfoil, is placed in such a wind tunnel and a controlled air flow is applied. The aerodynamic noise generated on the airfoil can be measured with a microphone array.

## References
[Hoeltgen, L., Breuß, M., Herold, G., & Sarradj, E. (2016). Sparse l1 Regularisation of Matrix Valued Models for Acoustic Source Characterisation. arXiv preprint arXiv:1607.00171.](https://arxiv.org/abs/1607.00171)

[Yardibi, T., Li, J., Stoica, P., & Cattafesta, L. N. (2008). Sparsity constrained deconvolution approaches for acoustic source mapping. Journal of the Acoustical Society of America, 123(5), 2631–2642.](http://dx.doi.org.proxy.findit.dtu.dk/10.1121/1.2896754)